In [ ]:
import torch as t
import numpy as np
import six
import torch.nn.functional as F
%matplotlib inline
import matplotlib.pyplot as plt

from lib.model.resnet import resnet50

def FindProposal(init_img,n_grids=3):

    n_mask = n_grids ** 2
    step = int(224 / n_grids)
    model = resnet50(pretrained=True).cuda()
    result_prob = []
    result_labels = []
    result_cord = []
    for ii in range(n_grids):#yi
        for jj in range(n_grids):#xi
            image = init_img.clone()
            for ch in range(3):
                image[ch,ii * step:(ii + 1) * step, jj * step:(jj + 1) * step] = t.from_numpy(np.array(0))

            image = image.unsqueeze(0).cuda()
            logit = model(image)#1*1000

            logit = logit.detach().squeeze().cpu()
            #logit = F.softmax(logit,0)
            _, index = logit.sort(0,True)

            index = np.array(index)
            logit = np.array(logit)
            result_prob.append(logit)

            result_labels.append(index[0])#最可能的label
            center_cor = [ii*step+step/2,jj*step + step/2]#y,x
            result_cord.append(center_cor)
    result_cord = np.array(result_cord)
    result_prob = np.array(result_prob)
    print(result_labels)
    #result_labels = np.array(result_labels)
    from collections import Counter
    word_counts = Counter(result_labels)
    # 出现频率最高的3个单词
    top_label = word_counts.most_common(5)
    print(top_label)
    prob_vector = result_prob[:,top_label[0][0]]
    print("第一次的概率为")
    print(prob_vector)
    print(prob_vector.shape)
    idx = np.argsort(prob_vector,axis=0)#升序
    print("coordi is {}".format(result_cord))
    choosed_box = result_cord[idx[0]]#中心点坐标
    print("we choose {}".format(choosed_box))
    #回归
    anchor = generate_anchor_base(choosed_box)
    print("anchor {}".format(anchor))

    probs = []
    labels = []
    coordinates =[]
    s = anchor.shape[0]
    for ii in range(0,s):
        image = init_img.clone()

        for ch in range(3):
            image[ch,int(anchor[ii][0]):int(anchor[ii][2]),int(anchor[ii][1]):int(anchor[ii][3])] = t.from_numpy(np.array(0))
        image = image.unsqueeze(0).cuda()
        logit = model(image)
        #logit = F.softmax(logit,1)
        logit = logit.detach().squeeze()
        _ , index = logit.sort(0,True)
        index = np.array(index)
        logit = np.array(logit)
        probs.append(logit)
        labels.append(index[0])
    probs = np.array(probs)
    labels = np.array(labels)
    print(labels)
    countlabel = Counter(labels)
    toplabel = countlabel.most_common(3)
    prob_target = probs[:,toplabel[0][0]]
    print("target probs is {}".format(prob_target))
    indx = np.argsort(prob_target,0)
    print("soted result {}".format(prob_target[indx]))
    target_box = anchor[indx[0]]
    print("target box {}".format(target_box))
    return target_box

def generate_anchor_base(zuobiao,base_size=2, ratios=[0.1,0.5,1,2],
                         anchor_scales=[0.1,0.5,1,1.5,2,5,8,10,12,15,18,20]):
    """Generate anchor base windows by enumerating aspect ratio and scales.

    Generate anchors that are scaled and modified to the given aspect ratios.
    Area of a scaled anchor is preserved when modifying to the given aspect
    ratio.

    :obj:`R = len(ratios) * len(anchor_scales)` anchors are generated by this
    function.
    The :obj:`i * len(anchor_scales) + j` th anchor corresponds to an anchor
    generated by :obj:`ratios[i]` and :obj:`anchor_scales[j]`.

    For example, if the scale is :math:`8` and the ratio is :math:`0.25`,
    the width and the height of the base window will be stretched by :math:`8`.
    For modifying the anchor to the given aspect ratio,
    the height is halved and the width is doubled.

    Args:
        base_size (number): The width and the height of the reference window.
        ratios (list of floats): This is ratios of width to height of
            the anchors.
        anchor_scales (list of numbers): This is areas of anchors.
            Those areas will be the product of the square of an element in
            :obj:`anchor_scales` and the original area of the reference
            window.

    Returns:
        ~numpy.ndarray:
        An array of shape :math:`(R, 4)`.
        Each element is a set of coordinates of a bounding box.
        The second axis corresponds to
        :math:`(y_{min}, x_{min}, y_{max}, x_{max})` of a bounding box.

    """
    py = zuobiao[0]
    px = zuobiao[1]

    anchor_base = np.zeros((len(ratios) * len(anchor_scales), 4),
                           dtype=np.float32)
    for i in six.moves.range(len(ratios)):
        for j in six.moves.range(len(anchor_scales)):
            h = base_size * anchor_scales[j] * np.sqrt(ratios[i])
            w = base_size * anchor_scales[j] * np.sqrt(1. / ratios[i])

            index = i * len(anchor_scales) + j
            anchor_base[index, 0] = py - h / 2.
            anchor_base[index, 1] = px - w / 2.
            anchor_base[index, 2] = py + h / 2.
            anchor_base[index, 3] = px + w / 2.
    return anchor_base#2*3=6个基础anchor
